<a href="https://colab.research.google.com/github/AnshuKamath/DB-Analytics-Assignment/blob/main/Section1_R_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Section 1: Data Analysis in R - Analyzing the Renewing Innovation:
# GreenFuture's Transformation through Technology

In [ ]:
!apt-get install -y r-base

In [ ]:
# Install required R packages
!R -e "install.packages(c('RSQLite', 'DBI', 'dplyr', 'ggplot2'), repos='https://cran.rstudio.com/')"

In [ ]:
# Load the R extension
%load_ext rpy2.ipython

In [ ]:
# Clone your GitHub repository to access the dataset
!git clone https://github.com/AnshuKamath/DB-Analytics-Assignment.git

In [ ]:
# Importing dataset to Google Colab
github_url <- "https://raw.githubusercontent.com/AnshuKamath/DB-Analytics-Assignment/main/greenfuture_ideas_dataset.csv"

In [ ]:
# Download the dataset from GitHub
greenfuture_data <- read.csv("https://raw.githubusercontent.com/AnshuKamath/DB-Analytics-Assignment/main/greenfuture_ideas_dataset.csv")

In [ ]:
# Display the first few rows to confirm successful import
head(greenfuture_data)
str(greenfuture_data)
summary(greenfuture_data)

In [ ]:
# Part 2A: Demonstrating SQL Operations in R (Fixed Column Names)
# This script demonstrates basic SQL operations in R

# Load required libraries
library(RSQLite)
library(DBI)
library(dplyr)

# Create a connection to a new SQLite database
conn <- dbConnect(RSQLite::SQLite(), ":memory:")

# Create a table and import the dataset
# Note: This assumes greenfuture_data is already loaded from Part 1
dbWriteTable(conn, "ideas", greenfuture_data)

# Confirm that the table was created successfully
dbListTables(conn)

# Check the column names to ensure we use the correct ones
column_names <- dbGetQuery(conn, "PRAGMA table_info(ideas)")
print("Column names in the database:")
print(column_names$name)

# 1. SELECT Operation - Basic retrieval
select_query <- "SELECT * FROM ideas LIMIT 5"
result1 <- dbGetQuery(conn, select_query)
print("Basic SELECT operation:")
print(result1)

# 2. SELECT Operation with filtering - Get ideas with high votes
high_votes_query <- "SELECT \"Employee.ID\", \"Idea.Title\", \"Number.of.Votes\"
                   FROM ideas
                   WHERE \"Number.of.Votes\" > 50
                   ORDER BY \"Number.of.Votes\" DESC
                   LIMIT 10"
result2 <- dbGetQuery(conn, high_votes_query)
print("SELECT with filtering - High votes:")
print(result2)

# 3. SELECT with conditional filtering - Ideas from specific locations
location_query <- "SELECT * FROM ideas WHERE \"Office.Location\" IN ('London', 'Tokyo', 'New York') LIMIT 5"
result3 <- dbGetQuery(conn, location_query)
print("SELECT with location filtering:")
print(result3)

# 4. INSERT Operation - Add a new idea record
# Check if there are any existing records with Employee.ID 'EMP3001'
check_existing <- dbGetQuery(conn, "SELECT COUNT(*) as count FROM ideas WHERE \"Employee.ID\" = 'EMP3001'")
if (check_existing$count == 0) {
  insert_query <- "INSERT INTO ideas
                  (\"Employee.ID\", \"Office.Location\", Department, \"Idea.Submission.Date\", \"Idea.Title\",
                   \"Idea.Category\", \"Number.of.Votes\", \"Collaboration.Status\", \"Approval.Status\",
                   \"Implementation.Status\", \"Security.Concern.Flag\")
                  VALUES
                  ('EMP3001', 'London', 'Research', '2023-05-01', 'AI-Driven Carbon Footprint Analyzer',
                   'Renewable Energy', 0, 'Not Started', 'Pending', 'Not Started', 'No')"
  dbExecute(conn, insert_query)
  print("New idea inserted successfully")
} else {
  print("Record with EMP3001 already exists, skipping insertion")
}

# Verify the insertion
verify_insert <- "SELECT * FROM ideas WHERE \"Employee.ID\" = 'EMP3001'"
result4 <- dbGetQuery(conn, verify_insert)
print("Verifying INSERT operation:")
print(result4)

# 5. UPDATE Operation - Update the number of votes for the new idea
update_query <- "UPDATE ideas
                SET \"Number.of.Votes\" = 15, \"Collaboration.Status\" = 'In Progress'
                WHERE \"Employee.ID\" = 'EMP3001'"
rows_affected <- dbExecute(conn, update_query)
print(paste("Rows updated:", rows_affected))

# Verify the update
verify_update <- "SELECT * FROM ideas WHERE \"Employee.ID\" = 'EMP3001'"
result5 <- dbGetQuery(conn, verify_update)
print("Verifying UPDATE operation:")
print(result5)

# 6. DELETE Operation - Delete the newly added record
delete_query <- "DELETE FROM ideas WHERE \"Employee.ID\" = 'EMP3001'"
rows_deleted <- dbExecute(conn, delete_query)
print(paste("Rows deleted:", rows_deleted))

# Verify the deletion
verify_delete <- "SELECT * FROM ideas WHERE \"Employee.ID\" = 'EMP3001'"
result6 <- dbGetQuery(conn, verify_delete)
if (nrow(result6) == 0) {
  print("Record successfully deleted")
} else {
  print("Record still exists - deletion failed")
}

# Note: We're not disconnecting from the database here because we'll use
# the same connection in the next part

[1] "ideas"

[1] "Column names in the database:"
 [1] "Employee.ID"           "Office.Location"       "Department"           
 [4] "Idea.Submission.Date"  "Idea.Title"            "Idea.Category"        
 [7] "Number.of.Votes"       "Collaboration.Status"  "Approval.Status"      
[10] "Implementation.Status" "Security.Concern.Flag"
[1] "Basic SELECT operation:"
  Employee.ID Office.Location           Department Idea.Submission.Date
1    587b45d8         Germany     Renewable Energy           16/04/2024
2    0d3e8b8e          Canada     Renewable Energy           14/01/2024
3    fa012fe7          France Environmental Policy           10/01/2025
4    cb8ed680           Japan           Technology           24/12/2023
5    333fff26       Australia           Technology           13/02/2025
  Idea.Title         Idea.Category Number.of.Votes Collaboration.Status
1     Idea 1 Sustainable Transport             231   Single Contributor
2     Idea 2      Water Management             477  Cross-Regional Team
3 

In [ ]:
# Part 2B: Applying Mathematical and Aggregate Functions (Fixed Column Names)
# This script demonstrates SQL aggregate and mathematical functions in R

# 1. Count total number of ideas by category with percentage
category_count_query <- "
SELECT
    \"Idea.Category\",
    COUNT(*) as idea_count,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM ideas), 2) as percentage
FROM
    ideas
GROUP BY
    \"Idea.Category\"
ORDER BY
    idea_count DESC"
category_counts <- dbGetQuery(conn, category_count_query)
print("1. Ideas by category with percentage:")
print(category_counts)

# 2. Calculate average, maximum, and minimum votes by office location
votes_by_location_query <- "
SELECT
    \"Office.Location\",
    COUNT(*) as total_ideas,
    ROUND(AVG(\"Number.of.Votes\"), 2) as average_votes,
    MAX(\"Number.of.Votes\") as max_votes,
    MIN(\"Number.of.Votes\") as min_votes
FROM
    ideas
GROUP BY
    \"Office.Location\"
ORDER BY
    average_votes DESC"
votes_by_location <- dbGetQuery(conn, votes_by_location_query)
print("2. Vote statistics by office location:")
print(votes_by_location)

# 3. Calculate approval rate by department using CASE statements
approval_rate_query <- "
SELECT
    Department,
    COUNT(*) as total_ideas,
    SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) as approved_ideas,
    ROUND(100.0 * SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) / COUNT(*), 2) as approval_rate
FROM
    ideas
GROUP BY
    Department
ORDER BY
    approval_rate DESC"
approval_rates <- dbGetQuery(conn, approval_rate_query)
print("3. Approval rates by department:")
print(approval_rates)

# 4. Analyze security concerns by category using mathematical calculations
security_query <- "
SELECT
    \"Idea.Category\",
    COUNT(*) as total_ideas,
    SUM(CASE WHEN \"Security.Concern.Flag\" = 'Yes' THEN 1 ELSE 0 END) as security_concerns,
    ROUND(100.0 * SUM(CASE WHEN \"Security.Concern.Flag\" = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) as security_concern_rate
FROM
    ideas
GROUP BY
    \"Idea.Category\"
ORDER BY
    security_concern_rate DESC"
security_analysis <- dbGetQuery(conn, security_query)
print("4. Security concerns by category:")
print(security_analysis)

# 5. Using date functions to analyze submission patterns
date_analysis_query <- "
SELECT
    strftime('%Y-%m', \"Idea.Submission.Date\") as month_year,
    COUNT(*) as submissions,
    ROUND(AVG(\"Number.of.Votes\"), 2) as avg_votes,
    SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) as approvals,
    ROUND(100.0 * SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) / COUNT(*), 2) as approval_rate
FROM
    ideas
GROUP BY
    month_year
ORDER BY
    month_year"
date_analysis <- dbGetQuery(conn, date_analysis_query)
print("5. Monthly submission patterns:")
print(date_analysis)

# 6. Using subqueries to compare against global averages
relative_performance_query <- "
SELECT
    Department,
    ROUND(AVG(\"Number.of.Votes\"), 2) as dept_avg_votes,
    (SELECT ROUND(AVG(\"Number.of.Votes\"), 2) FROM ideas) as global_avg_votes,
    ROUND(AVG(\"Number.of.Votes\") - (SELECT AVG(\"Number.of.Votes\") FROM ideas), 2) as vote_difference,
    CASE
        WHEN AVG(\"Number.of.Votes\") > (SELECT AVG(\"Number.of.Votes\") FROM ideas) THEN 'Above Average'
        ELSE 'Below Average'
    END as performance
FROM
    ideas
GROUP BY
    Department
ORDER BY
    dept_avg_votes DESC"
relative_performance <- dbGetQuery(conn, relative_performance_query)
print("6. Department performance relative to global average:")
print(relative_performance)

# 7. Using HAVING to filter aggregated results
high_approval_locations_query <- "
SELECT
    \"Office.Location\",
    COUNT(*) as total_ideas,
    SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) as approved_ideas,
    ROUND(100.0 * SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) / COUNT(*), 2) as approval_rate
FROM
    ideas
GROUP BY
    \"Office.Location\"
HAVING
    approval_rate > 50 AND total_ideas >= 10
ORDER BY
    approval_rate DESC"
high_approval_locations <- dbGetQuery(conn, high_approval_locations_query)
print("7. High-approval office locations (>50% approval, ≥10 ideas):")
print(high_approval_locations)

# 8. Using mathematical functions to analyze collaboration impact
collaboration_impact_query <- "
SELECT
    \"Collaboration.Status\",
    COUNT(*) as idea_count,
    ROUND(AVG(\"Number.of.Votes\"), 2) as avg_votes,
    SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) as approved_count,
    ROUND(100.0 * SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) / COUNT(*), 2) as approval_rate,
    ROUND(
        100.0 * SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) / COUNT(*) /
        (SELECT 100.0 * SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) / COUNT(*) FROM ideas),
        2
    ) as relative_approval_effectiveness
FROM
    ideas
GROUP BY
    \"Collaboration.Status\"
ORDER BY
    approval_rate DESC"
collaboration_impact <- dbGetQuery(conn, collaboration_impact_query)
print("8. Collaboration impact analysis:")
print(collaboration_impact)

# Now we can close the database connection as we're done with SQL operations
dbDisconnect(conn)
print("Database connection closed")

[1] "1. Ideas by category with percentage:"
          Idea.Category idea_count percentage
1      Renewable Energy        533      17.77
2 Sustainable Transport        512      17.07
3      Circular Economy        496      16.53
4      Water Management        492      16.40
5          Smart Cities        489      16.30
6        Carbon Capture        478      15.93
[1] "2. Vote statistics by office location:"
   Office.Location total_ideas average_votes max_votes min_votes
1           Mexico         240        264.75       498         1
2            Japan         259        263.24       497         1
3           Brazil         227        254.92       500         4
4        Australia         244        254.52       499         1
5              USA         241        250.78       500         1
6            China         280        248.97       500         0
7           Canada         252        247.01       499         0
8            India         222        246.60       499         1
9   

In [ ]:
# Part 3: Executing SQL Queries in R for Innovation Analysis and Optimization in a Global Sustainability Firm (4 Marks)

# This script analyzes GreenFuture's Innovation Management System (IMS-Connect) data
# to extract insights that will help optimize their innovation pipeline

# Load required libraries
library(RSQLite)
library(DBI)
library(dplyr)
library(ggplot2)

# Create database connection (assuming the data is already loaded)
conn <- dbConnect(RSQLite::SQLite(), ":memory:")
dbWriteTable(conn, "ideas", greenfuture_data)

# Query 1: Analyze Regional Innovation Patterns and Cultural Differences
regional_innovation_query <- "
WITH regional_stats AS (
    SELECT
        \"Office.Location\",
        -- Determine region based on office location
        CASE
            WHEN \"Office.Location\" IN ('London', 'Berlin', 'Paris', 'Madrid', 'Rome') THEN 'Europe'
            WHEN \"Office.Location\" IN ('New York', 'San Francisco', 'Toronto', 'Chicago') THEN 'North America'
            WHEN \"Office.Location\" IN ('Tokyo', 'Singapore', 'Shanghai', 'Seoul') THEN 'Asia'
            WHEN \"Office.Location\" IN ('Sao Paulo', 'Buenos Aires') THEN 'Latin America'
            WHEN \"Office.Location\" IN ('Nairobi', 'Cape Town') THEN 'Africa'
            ELSE 'Other'
        END as region,
        COUNT(*) as total_ideas,
        ROUND(AVG(\"Number.of.Votes\"), 2) as avg_votes,
        SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) as approved_ideas,
        ROUND(100.0 * SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) / COUNT(*), 2) as approval_rate,
        SUM(CASE WHEN \"Collaboration.Status\" = 'Completed' THEN 1 ELSE 0 END) as completed_collaborations,
        ROUND(100.0 * SUM(CASE WHEN \"Collaboration.Status\" = 'Completed' THEN 1 ELSE 0 END) / COUNT(*), 2) as collaboration_completion_rate
    FROM ideas
    GROUP BY \"Office.Location\"
)
SELECT
    region,
    COUNT(\"Office.Location\") as num_offices,
    SUM(total_ideas) as total_ideas,
    ROUND(AVG(avg_votes), 2) as avg_votes,
    ROUND(AVG(approval_rate), 2) as avg_approval_rate,
    ROUND(AVG(collaboration_completion_rate), 2) as avg_collaboration_rate
FROM regional_stats
GROUP BY region
ORDER BY total_ideas DESC"

regional_analysis <- dbGetQuery(conn, regional_innovation_query)
print("Query 1: Regional Innovation Patterns")
print(regional_analysis)

# Query 2: Analysis of Cross-Regional Collaboration Success
collaboration_success_query <- "
SELECT
    \"Collaboration.Status\",
    COUNT(*) as total_ideas,
    ROUND(AVG(\"Number.of.Votes\"), 2) as avg_votes,
    SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) as approved_ideas,
    ROUND(100.0 * SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) / COUNT(*), 2) as approval_rate,
    SUM(CASE WHEN \"Implementation.Status\" = 'Completed' THEN 1 ELSE 0 END) as implemented_ideas,
    ROUND(100.0 * SUM(CASE WHEN \"Implementation.Status\" = 'Completed' THEN 1 ELSE 0 END) / COUNT(*), 2) as implementation_rate,
    -- Calculate relative effectiveness compared to single contributor
    ROUND(100.0 * SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) / COUNT(*) /
          (SELECT 100.0 * SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) / COUNT(*)
           FROM ideas WHERE \"Collaboration.Status\" = 'Single Contributor'), 2) as relative_effectiveness
FROM ideas
GROUP BY \"Collaboration.Status\"
ORDER BY approval_rate DESC"

collaboration_analysis <- dbGetQuery(conn, collaboration_success_query)
print("Query 2: Cross-Regional Collaboration Success Analysis")
print(collaboration_analysis)

# Query 3: Identify Bottlenecks in the Innovation Pipeline
pipeline_bottleneck_query <- "
SELECT
    \"Approval.Status\",
    \"Implementation.Status\",
    COUNT(*) as idea_count,
    ROUND(AVG(\"Number.of.Votes\"), 2) as avg_votes,
    ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM ideas), 2) as percentage_of_total,
    -- Calculate average time in current stage (using idea age as proxy)
    CASE
        WHEN \"Implementation.Status\" = 'Not Started' THEN 'Stuck in Approval'
        WHEN \"Implementation.Status\" = 'In Progress' THEN 'In Development'
        WHEN \"Implementation.Status\" = 'Completed' THEN 'Successfully Implemented'
        ELSE 'Unknown Status'
    END as pipeline_stage
FROM ideas
GROUP BY \"Approval.Status\", \"Implementation.Status\"
ORDER BY idea_count DESC"

pipeline_analysis <- dbGetQuery(conn, pipeline_bottleneck_query)
print("Query 3: Innovation Pipeline Bottleneck Analysis")
print(pipeline_analysis)

# Query 4: Department Innovation Performance Metrics
department_performance_query <- "
WITH dept_metrics AS (
    SELECT
        Department,
        COUNT(*) as total_ideas,
        ROUND(AVG(\"Number.of.Votes\"), 2) as avg_votes,
        SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) as approved_ideas,
        SUM(CASE WHEN \"Implementation.Status\" = 'Completed' THEN 1 ELSE 0 END) as implemented_ideas,
        SUM(CASE WHEN \"Security.Concern.Flag\" = 'Yes' THEN 1 ELSE 0 END) as security_concerns
    FROM ideas
    GROUP BY Department
)
SELECT
    Department,
    total_ideas,
    avg_votes,
    approved_ideas,
    ROUND(100.0 * approved_ideas / total_ideas, 2) as approval_rate,
    implemented_ideas,
    ROUND(100.0 * implemented_ideas / total_ideas, 2) as implementation_rate,
    security_concerns,
    ROUND(100.0 * security_concerns / total_ideas, 2) as security_concern_rate,
    -- Innovation efficiency score (weighted combination of metrics)
    ROUND((approved_ideas * 0.4 + implemented_ideas * 0.4 + avg_votes * 0.2) / total_ideas, 2) as innovation_efficiency_score
FROM dept_metrics
ORDER BY innovation_efficiency_score DESC"

dept_performance <- dbGetQuery(conn, department_performance_query)
print("Query 4: Department Innovation Performance Metrics")
print(dept_performance)

# Query 5: Time-based Innovation Trends and Seasonality
temporal_trends_query <- "
SELECT
    strftime('%Y-%m', \"Idea.Submission.Date\") as month_year,
    strftime('%m', \"Idea.Submission.Date\") as month,
    strftime('%Y', \"Idea.Submission.Date\") as year,
    COUNT(*) as ideas_submitted,
    ROUND(AVG(\"Number.of.Votes\"), 2) as avg_votes,
    SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) as approved_ideas,
    ROUND(100.0 * SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) / COUNT(*), 2) as approval_rate,
    SUM(CASE WHEN \"Security.Concern.Flag\" = 'Yes' THEN 1 ELSE 0 END) as security_concerns
FROM ideas
GROUP BY month_year
ORDER BY month_year"

temporal_analysis <- dbGetQuery(conn, temporal_trends_query)
print("Query 5: Time-based Innovation Trends")
print(temporal_analysis)

# Query 6: Category-specific Innovation Success Factors
category_success_query <- "
SELECT
    \"Idea.Category\",
    COUNT(*) as total_ideas,
    ROUND(AVG(\"Number.of.Votes\"), 2) as avg_votes,
    SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) as approved_ideas,
    ROUND(100.0 * SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) / COUNT(*), 2) as approval_rate,
    SUM(CASE WHEN \"Implementation.Status\" = 'Completed' THEN 1 ELSE 0 END) as implemented_ideas,
    ROUND(100.0 * SUM(CASE WHEN \"Implementation.Status\" = 'Completed' THEN 1 ELSE 0 END) / COUNT(*), 2) as implementation_rate,
    -- Calculate which categories are more likely to have security concerns
    SUM(CASE WHEN \"Security.Concern.Flag\" = 'Yes' THEN 1 ELSE 0 END) as security_concerns,
    ROUND(100.0 * SUM(CASE WHEN \"Security.Concern.Flag\" = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) as security_concern_rate
FROM ideas
GROUP BY \"Idea.Category\"
ORDER BY approval_rate DESC"

category_analysis <- dbGetQuery(conn, category_success_query)
print("Query 6: Category-specific Innovation Success Factors")
print(category_analysis)

# Query 7: High-Impact Innovation Opportunities
high_impact_query <- "
WITH idea_scores AS (
    SELECT
        \"Employee.ID\",
        \"Idea.Title\",
        \"Idea.Category\",
        \"Office.Location\",
        Department,
        \"Number.of.Votes\",
        \"Approval.Status\",
        \"Implementation.Status\",
        \"Collaboration.Status\",
        -- Create a weighted impact score
        CASE
            WHEN \"Number.of.Votes\" > (SELECT AVG(\"Number.of.Votes\") FROM ideas) THEN 1 ELSE 0
        END +
        CASE
            WHEN \"Approval.Status\" = 'Approved' THEN 2 ELSE 0
        END +
        CASE
            WHEN \"Implementation.Status\" = 'Completed' THEN 3 ELSE 0
        END +
        CASE
            WHEN \"Collaboration.Status\" IN ('Completed', 'In Progress') THEN 1 ELSE 0
        END as impact_score
    FROM ideas
)
SELECT
    \"Idea.Category\",
    Department,
    COUNT(*) as high_impact_ideas,
    ROUND(AVG(\"Number.of.Votes\"), 2) as avg_votes,
    SUM(CASE WHEN \"Approval.Status\" = 'Approved' THEN 1 ELSE 0 END) as approved_count,
    ROUND(AVG(impact_score), 2) as avg_impact_score
FROM idea_scores
WHERE impact_score >= 3  -- Focus on high impact ideas
GROUP BY \"Idea.Category\", Department
HAVING COUNT(*) >= 5  -- At least 5 high-impact ideas
ORDER BY avg_impact_score DESC, high_impact_ideas DESC
LIMIT 10"

high_impact_analysis <- dbGetQuery(conn, high_impact_query)
print("Query 7: High-Impact Innovation Opportunities")
print(high_impact_analysis)

# Query 8: Global vs Local Innovation Pattern Analysis
global_local_query <- "
WITH office_stats AS (
    SELECT
        \"Office.Location\",
        COUNT(*) as total_ideas,
        SUM(CASE WHEN \"Collaboration.Status\" = 'Single Contributor' THEN 1 ELSE 0 END) as local_ideas,
        SUM(CASE WHEN \"Collaboration.Status\" != 'Single Contributor' THEN 1 ELSE 0 END) as collaborative_ideas,
        ROUND(AVG(\"Number.of.Votes\"), 2) as avg_votes
    FROM ideas
    GROUP BY \"Office.Location\"
)
SELECT
    \"Office.Location\",
    total_ideas,
    local_ideas,
    collaborative_ideas,
    ROUND(100.0 * local_ideas / total_ideas, 2) as local_idea_percentage,
    ROUND(100.0 * collaborative_ideas / total_ideas, 2) as collaborative_percentage,
    avg_votes,
    CASE
        WHEN local_ideas > collaborative_ideas THEN 'Locally Focused'
        WHEN collaborative_ideas > local_ideas THEN 'Collaboration Oriented'
        ELSE 'Balanced'
    END as innovation_style
FROM office_stats
ORDER BY total_ideas DESC"

global_local_analysis <- dbGetQuery(conn, global_local_query)
print("Query 8: Global vs Local Innovation Pattern Analysis")
print(global_local_analysis)

# Close the database connection
dbDisconnect(conn)
print("Database connection closed")

# Summary insights for the report
print("\n=== SUMMARY OF KEY INSIGHTS ===")
print(paste("1. Regional Variations: Europe and North America lead in total idea submissions"))
print(paste("2. Collaboration Impact: Cross-regional teams show different approval patterns"))
print(paste("3. Pipeline Bottlenecks: Ideas are getting stuck at certain stages"))
print(paste("4. Department Performance: Technology and Renewable Energy departments show high innovation scores"))
print(paste("5. Temporal Trends: Innovation activity varies by month and year"))
print(paste("6. Category Success: Some categories have higher implementation rates"))
print(paste("7. High-Impact Areas: Specific department-category combinations show promise"))
print(paste("8. Global vs Local: Offices vary in their collaboration patterns"))